sequance to sequance  language translator from english to arabic using RNN seq2seq modeling

import important libraries

In [1]:
import numpy as np
import keras
import os
from pathlib import Path


intialize global variables

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "/content/ara_.txt"

open data path

In [3]:
with open(data_path,"r",encoding="utf-8") as f:
  lines=f.read().split("\n")
  print(len(lines))
  for i in range (5):
    print(lines[i])

10743
Hi.	مرحبًا.
Run!	اركض!
Help!	النجدة!
Jump!	اقفز!
Stop!	قف!


data vectorization

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    parts = line.split("\t")
    if len(parts) >= 2:
        input_text, target_text = parts[:2]
        target_text = "\t" + target_text + "\n"
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    else:
        print(f"Ignoring line: {line}")

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) #unique car in english
num_decoder_tokens = len(target_characters) #unique car in arabic
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length) #max num of english char
print("Max sequence length for outputs:", max_decoder_seq_length)



Number of samples: 10000
Number of unique input tokens: 74
Number of unique output tokens: 106
Max sequence length for inputs: 47
Max sequence length for outputs: 67


show input and output

In [5]:
target_text

'\tالحفل اختتم بالنشيد الوطني .\n'

In [6]:
input_characters

[' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '’',
 '€']

In [7]:
input_text

'The concert concluded with the national anthem.'

In [8]:
[len(txt) for txt in input_texts]

[3,
 4,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,

enumeration to return charachter and its index.

In [9]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) #key=char , value=index
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), #num_encoder_tokens=unique car in english
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), # input is num of arabic words
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)



In [10]:
encoder_input_data.shape #rows,columns,max num of char in e english

(10000, 47, 74)

In [11]:
input_token_index['H']

27

In [12]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 ',': 4,
 '-': 5,
 '.': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 ';': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'X': 43,
 'Y': 44,
 'Z': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71,
 '’': 72,
 '€': 73}

one hot encoding

In [13]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [14]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 ',': 4,
 '-': 5,
 '.': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 ';': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'X': 43,
 'Y': 44,
 'Z': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71,
 '’': 72,
 '€': 73}

loop to print input data (char)

In [15]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  print(input_text)
  for t, char in enumerate(input_text):
    #print(t,char)
    encoder_input_data[i, t, input_token_index[char]] = 1.0 #go to the index for each car in row and column and put 1


Streaming output truncated to the last 5000 lines.
Cut the cake with a knife.
Darwin changed everything.
Did you go out last night?
Do you have a credit card?
Do you have a larger size?
Do you have a larger size?
Do you have any sunscreen?
Do you know what happened?
Do you know where Tom hid?
Do you like San Francisco?
Do you think Tom saw Mary?
Does anyone speak English?
Does he have a girlfriend?
Does he know what you did?
Does your friend like tea?
Don't forget to add me in.
Don't give up if you fail.
Don't hang up yet, please.
Even Homer sometimes nods.
Even Homer sometimes nods.
Even Homer sometimes nods.
Everybody agrees with you.
Everybody agrees with you.
Everybody had a good year.
Everyone is still smiling.
Forgive me for being late.
Give me a definite answer.
Hand me that book, please.
He agrees with my opinion.
He always tells the truth.
He always tells the truth.
He always tells the truth.
He bought a dress for her.
He came back from America.
He couldn't run very fast.
He i

In [16]:
for t,char in enumerate(input_text):
  print(char)

T
h
e
 
c
o
n
c
e
r
t
 
c
o
n
c
l
u
d
e
d
 
w
i
t
h
 
t
h
e
 
n
a
t
i
o
n
a
l
 
a
n
t
h
e
m
.


Build our model

In [17]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

compile our model

In [18]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")

Epoch 1/100
125/125 [==============================] - 7s 28ms/step - loss: 1.2324 - accuracy: 0.7433 - val_loss: 1.5617 - val_accuracy: 0.6158
Epoch 2/100
125/125 [==============================] - 2s 20ms/step - loss: 0.9791 - accuracy: 0.7531 - val_loss: 1.5306 - val_accuracy: 0.6228
Epoch 3/100
125/125 [==============================] - 2s 16ms/step - loss: 0.9124 - accuracy: 0.7649 - val_loss: 1.4559 - val_accuracy: 0.6235
Epoch 4/100
125/125 [==============================] - 2s 14ms/step - loss: 0.8461 - accuracy: 0.7778 - val_loss: 1.3110 - val_accuracy: 0.6527
Epoch 5/100
125/125 [==============================] - 2s 14ms/step - loss: 0.7983 - accuracy: 0.7892 - val_loss: 1.2568 - val_accuracy: 0.6699
Epoch 6/100
125/125 [==============================] - 2s 14ms/step - loss: 0.7703 - accuracy: 0.7955 - val_loss: 1.2341 - val_accuracy: 0.6757
Epoch 7/100
125/125 [==============================] - 2s 15ms/step - loss: 0.7503 - accuracy: 0.7997 - val_loss: 1.2036 - val_accuracy:

model evaluation

In [20]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

loop to evaluate

In [23]:
for seq_index in range(30):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: أنا أحبك.

-
Input sentence: Run!
Decoded sentence: هيا بنان.

-
Input sentence: Help!
Decoded sentence: انظر إلي

-
Input sentence: Jump!
Decoded sentence: انظر إلي

-
Input sentence: Stop!
Decoded sentence: الجو حار جداً.

-
Input sentence: Go on.
Decoded sentence: الجو دريس.

-
Input sentence: Go on.
Decoded sentence: الجو دريس.

-
Input sentence: Hello!
Decoded sentence: انظر إلي

-
Input sentence: Hurry!
Decoded sentence: الجو حار جداً.

-
Input sentence: Hurry!
Decoded sentence: الجو حار جداً.

-
Input sentence: I see.
Decoded sentence: أنا أحبك.

-
Input sentence: I won!
Decoded sentence: أنا في البيت

-
Input sentence: Smile.
Decoded sentence: هيا بنان.

-
Input sentence: Cheers!
Decoded sentence: انظر إلي

-
Input sentence: Got it?
Decoded sentence: الجو حار جداً.

-
Input sentence: He ran.
Decoded sentence: انظر إلي

-
Input sentence: I know.
Decoded sentence: أنا في البيت

-
Input sentence: I know.
Decoded sentence: أنا في البيت

-
Inp